# Steady-State Axthelm System with Built-in Newton Method

## Overview

This notebook implements the **steady-state** solution of Axthelm's pedestrian flow model using NGSolve's **built-in Newton solver** with automatic differentiation.

### System of Equations (Axthelm 2016)

Steady-state (ρₜ = 0):

```
(Helmholtz)   1/f²(ρ) ψ - ε₁² Δψ = 0

(Velocity)    u = -f(ρ) ∇Φ / √(|∇Φ|² + ε₂²),  Φ = -ε₁ ln(ψ)

(Continuity)  -ε₀ Δρ + ∇·(ρu) = 0
```

**Key approach**: Use symbolic trial functions to define coupled residual, let NGSolve compute Jacobian automatically.

---

## 1. Imports

In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton  # Built-in Newton solver
import numpy as np
import time

print("✓ Imports successful")

✓ Imports successful


## 2. Geometry and Mesh

In [2]:
# Domain parameters
Hwid = 1.0  # Width [m]
Hcol = 1.0  # Height [m]
mesh_maxh = 0.05  # Maximum mesh size [m]

# Create rectangle domain
rect = Rectangle(Hwid, Hcol).Face()

# Name boundaries
rect.edges.Min(X).name = "left"    # Left wall
rect.edges.Max(X).name = "right"   # Right wall
rect.edges.Max(Y).name = "entry"   # Entry (top)
rect.edges.Min(Y).name = "exit"    # Exit (bottom)

# Generate mesh
geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")
print(f"Boundaries: {mesh.GetBoundaries()}")
print("\n✓ Mesh generated")

Mesh: 514 vertices, 946 elements
Boundaries: ('exit', 'right', 'entry', 'left')

✓ Mesh generated


## 3. Parameters

### 3.1 Physical Parameters (Weidmann)

In [3]:
# Fundamental diagram (Weidmann)
u0 = 1.36          # Free-flow speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

print("Physical parameters:")
print(f"  u₀ = {u0} m/s")
print(f"  ρc = {rho_c} ped/m²")
print(f"  γ  = {gamma_w} ped/m²")

Physical parameters:
  u₀ = 1.36 m/s
  ρc = 8.0 ped/m²
  γ  = 1.913 ped/m²


### 3.2 Regularization Parameters

In [4]:
# Regularization (Axthelm)
eps_0 = 0.05       # Continuity diffusion [m²]
eps_1 = 0.1        # Helmholtz viscosity [m]
eps_2 = 0.01       # Velocity regularization [dimensionless]

print("\nRegularization parameters:")
print(f"  ε₀ = {eps_0} m² (continuity diffusion)")
print(f"  ε₁ = {eps_1} m (Helmholtz viscosity)")
print(f"  ε₂ = {eps_2} (velocity regularization)")


Regularization parameters:
  ε₀ = 0.05 m² (continuity diffusion)
  ε₁ = 0.1 m (Helmholtz viscosity)
  ε₂ = 0.01 (velocity regularization)


### 3.3 Boundary Conditions

In [5]:
# Inflow rate at entry
g_inflow = 0.5     # Influx [ped/(m·s)]

print("\nBoundary conditions:")
print(f"  Entry inflow: g = {g_inflow} ped/(m·s)")
print(f"  Exit potential: ψ = 1 (Dirichlet)")
print(f"  Walls: ∇ψ·ν = 0, ∇ρ·ν = 0 (natural)")


Boundary conditions:
  Entry inflow: g = 0.5 ped/(m·s)
  Exit potential: ψ = 1 (Dirichlet)
  Walls: ∇ψ·ν = 0, ∇ρ·ν = 0 (natural)


### 3.4 Numerical Parameters

In [6]:
p_order = 2           # Polynomial order
max_iter = 50         # Maximum Newton iterations
tol = 1e-6            # Convergence tolerance
dampfactor = 0.1      # Newton damping (0.1 = take smaller steps for stability)

print("\\nNumerical parameters:")
print(f"  Polynomial order: {p_order}")
print(f"  Max iterations: {max_iter}")
print(f"  Tolerance: {tol}")
print(f"  Damping factor: {dampfactor} (smaller = more stable)")
print("\\n✓ All parameters set")

\nNumerical parameters:
  Polynomial order: 2
  Max iterations: 50
  Tolerance: 1e-06
  Damping factor: 0.1 (smaller = more stable)
\n✓ All parameters set


## 4. Fundamental Diagram (Smooth Weidmann)

**Critical**: Must be smooth for automatic differentiation!

In [7]:
def f_smooth(rho):
    """
    Smooth Weidmann fundamental diagram for automatic differentiation.
    
    Returns walking speed f(ρ) [m/s] as function of density ρ [ped/m²].
    
    Uses smooth regularization to avoid:
    - Division by zero at ρ → 0
    - Non-differentiable IfPos constructs
    """
    rho_min = 0.01  # Minimum density for regularization
    
    # Smooth approximation: max(ρ, ρ_min) ≈ (ρ + √(ρ² + ρ_min²))/2
    rho_reg = (rho + sqrt(rho**2 + rho_min**2)) / 2
    
    # Weidmann formula
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    return speed

print("✓ Smooth Weidmann function defined")
print("  - No IfPos() calls")
print("  - Continuous derivatives")
print("  - Safe for automatic differentiation")

✓ Smooth Weidmann function defined
  - No IfPos() calls
  - Continuous derivatives
  - Safe for automatic differentiation


## 5. Finite Element Spaces

Create **compound space** for coupled (ρ, ψ) system.

In [8]:
# Separate spaces
fes_rho = H1(mesh, order=p_order)  # Density (no Dirichlet)
fes_psi = H1(mesh, order=p_order, dirichlet="exit")  # Potential (ψ=1 at exit)

# Compound space for (ρ, ψ)
fes = fes_rho * fes_psi

# Velocity space (for post-processing)
fes_u = H1(mesh, order=p_order, dim=2)

print("Finite element spaces:")
print(f"  Density (ρ):      {fes_rho.ndof} DOFs")
print(f"  Potential (ψ):    {fes_psi.ndof} DOFs")
print(f"  Coupled (ρ, ψ):   {fes.ndof} DOFs")
print(f"  Velocity (u):     {fes_u.ndof} DOFs")
print("\n✓ Finite element spaces created")

Finite element spaces:
  Density (ρ):      1973 DOFs
  Potential (ψ):    1973 DOFs
  Coupled (ρ, ψ):   3946 DOFs
  Velocity (u):     1973 DOFs

✓ Finite element spaces created


## 6. Residual Form

Define coupled residual R(ρ, ψ) = 0 for Newton solver.

**Key**: Velocity computed symbolically from trial functions!

In [9]:
print("Setting up coupled residual form...\\n")

# Trial and test functions for coupled system
(u_rho, u_psi), (v_w, v_phi) = fes.TnT()

print("Step 1: Computing symbolic velocity from trial functions")
# Compute ∇Φ from trial ψ
# Φ = -ε₁ ln(ψ)  →  ∇Φ = -ε₁ (∇ψ/ψ)
# 
# CRITICAL: Add regularization to avoid division by zero when ψ → 0
psi_reg = 1e-6  # Regularization parameter
grad_phi = -eps_1 * grad(u_psi) / (u_psi + psi_reg)

# Regularized norm: √(|∇Φ|² + ε₂²)
grad_phi_norm_sq = InnerProduct(grad_phi, grad_phi) + eps_2**2
grad_phi_norm = sqrt(grad_phi_norm_sq)

# Velocity: u = -f(ρ) ∇Φ / √(|∇Φ|² + ε₂²)
f_trial = f_smooth(u_rho)
u_trial = -f_trial * grad_phi / grad_phi_norm

print("  ✓ Symbolic velocity defined")
print("    u = -f(ρ) · ∇Φ / √(|∇Φ|² + ε₂²)")
print("    where ∇Φ = -ε₁ (∇ψ/(ψ + 10⁻⁶)) [regularized!]")

# Normal vector for boundary terms
n = specialcf.normal(2)

print("\\nStep 2: Assembling residual form")
# Create residual BilinearForm
R = BilinearForm(fes, symmetric=False)

# ============================================================
# HELMHOLTZ RESIDUAL: 1/f²(ρ) ψ - ε₁² Δψ = 0
# ============================================================
print("  Adding Helmholtz terms...")

# Zero-order term: ∫ (1/f²(ρ)) ψ φ dx
kappa_sq = 1.0 / (f_smooth(u_rho)**2)
R += kappa_sq * u_psi * v_phi * dx

# Laplacian term: ∫ ε₁² ∇ψ·∇φ dx (after integration by parts)
R += eps_1**2 * grad(u_psi) * grad(v_phi) * dx

# Boundary conditions (from Axthelm paper, eq 11-12):
# - ψ = 1 on Γexit (Dirichlet - handled by dirichlet=\"exit\" in fes_psi)
# - ∇ψ·ν = 0 on walls and entry (natural BC - automatically satisfied!)

print("    ✓ Helmholtz: ∫ κ²(ρ)ψφ + ε₁²∇ψ·∇φ dx")
print("    ✓ BC: ψ=1 at exit (Dirichlet), ∇ψ·ν=0 elsewhere (natural)")

# ============================================================
# CONTINUITY RESIDUAL: -ε₀ Δρ + ∇·(ρu) = 0
# ============================================================
print("  Adding Continuity terms...")

# Diffusion term: ∫ ε₀ ∇ρ·∇w dx (after integration by parts)
R += eps_0 * grad(u_rho) * grad(v_w) * dx

# Advection term: ∫ ∇·(ρu) w dx = -∫ (ρu)·∇w dx (integration by parts)
R += -u_rho * InnerProduct(u_trial, grad(v_w)) * dx

# Entry boundary condition: inflow rate
# ∫ g_inflow w ds (negative because it's a source)
R += -g_inflow * v_w * ds("entry")

# Boundary conditions (from Axthelm paper, eq 13-14):
# - ∇ρ·ν = 0 on all boundaries (natural - automatically satisfied)
# - Flow rate g_inflow at entry (added above as source term)

print("    ✓ Continuity: ∫ ε₀∇ρ·∇w - ρu·∇w dx - ∫ g w ds(entry)")
print("    ✓ BC: ∇ρ·ν=0 everywhere (natural), flux g at entry")

print("\\n✓ Coupled residual form assembled")
print("  → NGSolve will compute Jacobian via automatic differentiation")

Setting up coupled residual form...\n
Step 1: Computing symbolic velocity from trial functions
  ✓ Symbolic velocity defined
    u = -f(ρ) · ∇Φ / √(|∇Φ|² + ε₂²)
    where ∇Φ = -ε₁ (∇ψ/(ψ + 10⁻⁶)) [regularized!]
\nStep 2: Assembling residual form
  Adding Helmholtz terms...
    ✓ Helmholtz: ∫ κ²(ρ)ψφ + ε₁²∇ψ·∇φ dx
    ✓ BC: ψ=1 at exit (Dirichlet), ∇ψ·ν=0 elsewhere (natural)
  Adding Continuity terms...
    ✓ Continuity: ∫ ε₀∇ρ·∇w - ρu·∇w dx - ∫ g w ds(entry)
    ✓ BC: ∇ρ·ν=0 everywhere (natural), flux g at entry
\n✓ Coupled residual form assembled
  → NGSolve will compute Jacobian via automatic differentiation


## 7. Initial Guess

**Critical for Newton convergence!**

In [10]:
print("Setting initial guess...\\n")

# Grid function for coupled system
gf = GridFunction(fes, name="solution")
gf_rho, gf_psi = gf.components

# Initial density: uniform, moderate crowd density
rho_init = 1.0  # [ped/m²]
gf_rho.Set(rho_init)

# Initial potential: CRITICAL - ψ must stay > 0 everywhere!
# Since Φ = -ε₁ ln(ψ), and we divide by ψ in ∇Φ = -ε₁ (∇ψ/ψ)
# 
# Strategy: Use exponential profile that's always positive
# ψ(y) = exp((y - Hcol)/ε₁)  → ψ(entry)=exp(0)=1, ψ increases toward exit
# 
# Better: Scale so ψ=1 at exit (y=0)
# ψ(y) = exp(y/eps_1) at y=0 gives 1, at y=Hcol gives exp(Hcol/eps_1)
psi_init = exp(-y / eps_1)  # Decreases from entry to exit
gf_psi.Set(psi_init)

# Apply Dirichlet BC for ψ at exit
gf_psi.Set(1.0, definedon=mesh.Boundaries("exit"))

print(f"Initial guess:")
print(f"  ρ = {rho_init} ped/m² (uniform)")
print(f"  ψ = exp(-y/{eps_1}) (exponential, always positive!)")
print(f"  ψ(entry at y={Hcol}) ≈ {exp(-Hcol/eps_1):.4f}")
print(f"  ψ(exit at y=0) = 1.0")
print(f"\\nSolution vector statistics:")
print(f"  ρ: min = {min(gf_rho.vec):.4f}, max = {max(gf_rho.vec):.4f}")
print(f"  ψ: min = {min(gf_psi.vec):.4f}, max = {max(gf_psi.vec):.4f}")

# Safety check
psi_min = min(gf_psi.vec)
if psi_min <= 0:
    print(f"\\n⚠ WARNING: ψ_min = {psi_min} ≤ 0! This will cause division by zero!")
else:
    print(f"\\n✓ ψ > 0 everywhere (min = {psi_min:.6f})")
    
print("\\n✓ Initial guess set")

Setting initial guess...\n
Initial guess:
  ρ = 1.0 ped/m² (uniform)
  ψ = exp(-y/0.1) (exponential, always positive!)
  ψ(entry at y=1.0) ≈ 0.0000
  ψ(exit at y=0) = 1.0
\nSolution vector statistics:
  ρ: min = -0.0000, max = 1.0000
  ψ: min = -0.0000, max = 1.0000
\n⚠ WARNING: ψ_min = -6.45317133063372e-15 ≤ 0! This will cause division by zero!
\n✓ Initial guess set


## 8. Solve with Built-in Newton Method

Call NGSolve's automatic differentiation Newton solver.

In [11]:
print("="*60)
print("BUILT-IN NEWTON SOLVER")
print("="*60)
print(f"\\nSettings:")
print(f"  dampfactor: {dampfactor}")
print(f"  maxerr: {tol}")
print(f"  maxit: {max_iter}")
print(f"  inverse: sparsecholesky")
print(f"\\nStarting Newton solve...\\n")

t_start = time.time()

try:
    with TaskManager():
        Newton(
            a=R,                          # Residual form
            u=gf,                         # Solution vector
            freedofs=fes.FreeDofs(),      # Free DOFs (not Dirichlet)
            maxit=max_iter,               # Max iterations
            maxerr=tol,                   # Convergence tolerance
            inverse="sparsecholesky",     # Linear solver
            dampfactor=dampfactor,        # Damping factor (use variable!)
            printing=True                 # Print convergence
        )
    success = True
except Exception as e:
    print(f"\\n✗ Newton failed: {e}")
    success = False

t_end = time.time()
wall_time = t_end - t_start

print("\\n" + "="*60)
if success:
    print(f"✓ Newton solver converged successfully")
else:
    print(f"✗ Newton solver failed to converge")
print(f"  Wall time: {wall_time:.4f} s")
print("="*60)

BUILT-IN NEWTON SOLVER
\nSettings:
  dampfactor: 0.1
  maxerr: 1e-06
  maxit: 50
  inverse: sparsecholesky
\nStarting Newton solve...\n
Newton iteration  0
err =  2.620792222778525
Newton iteration  1
err =  2.3355373518210194
Newton iteration  2
err =  4.131520199166036
Newton iteration  3
err =  7.3129379565731725
Newton iteration  4
err =  9.336143241964995
Newton iteration  5
err =  19.438833476791856
Newton iteration  6
err =  34.28159655659515
Newton iteration  7
err =  49.85069116107101
Newton iteration  8
err =  595.4595474795672
Newton iteration  9
err =  983.1566271450342
Newton iteration  10
err =  3447.767957848954
Newton iteration  11
err =  14771.569272554074
Newton iteration  12
err =  316627.0362159483
Newton iteration  13
err =  68777.23536088463
Newton iteration  14
err =  nan
Newton iteration  15
err =  nan
Newton iteration  16
err =  nan
Newton iteration  17
err =  nan
Newton iteration  18
err =  nan
Newton iteration  19
err =  nan
Newton iteration  20
err =  nan
Ne

## 9. Post-Processing

### 9.1 Compute Velocity Field

In [12]:
print("Computing velocity field from solution...\n")

# Grid function for velocity
gf_u = GridFunction(fes_u, name="velocity")

# Compute velocity using converged solution
# u = -f(ρ) ∇Φ / √(|∇Φ|² + ε₂²)
grad_phi_cf = -eps_1 * grad(gf_psi) / gf_psi
grad_phi_norm_sq_cf = InnerProduct(grad_phi_cf, grad_phi_cf) + eps_2**2
grad_phi_norm_cf = sqrt(grad_phi_norm_sq_cf)

speed_cf = f_smooth(gf_rho)
u_cf = -speed_cf * grad_phi_cf / grad_phi_norm_cf

gf_u.Set(u_cf)

print("✓ Velocity field computed")

Computing velocity field from solution...

✓ Velocity field computed


### 9.2 Solution Statistics

In [13]:
print("\n" + "="*60)
print("SOLUTION STATISTICS")
print("="*60)

# Domain area
domain_area = Integrate(1.0, mesh)

# Density statistics
rho_integral = Integrate(gf_rho, mesh)
rho_mean = rho_integral / domain_area

# Interpolate to P1 for min/max
fes_p1 = H1(mesh, order=1)
gf_rho_p1 = GridFunction(fes_p1)
gf_psi_p1 = GridFunction(fes_p1)
gf_rho_p1.Set(gf_rho)
gf_psi_p1.Set(gf_psi)

print(f"\nDensity (ρ):")
print(f"  integral = {rho_integral:.6f} ped")
print(f"  mean = {rho_mean:.6f} ped/m²")
print(f"  min ≈ {min(gf_rho_p1.vec):.6f} ped/m²")
print(f"  max ≈ {max(gf_rho_p1.vec):.6f} ped/m²")

print(f"\nPotential (ψ):")
print(f"  min ≈ {min(gf_psi_p1.vec):.6f}")
print(f"  max ≈ {max(gf_psi_p1.vec):.6f}")

print(f"\nDomain area: {domain_area:.6f} m²")
print("="*60)


SOLUTION STATISTICS

Density (ρ):
  integral = nan ped
  mean = nan ped/m²
  min ≈ nan ped/m²
  max ≈ nan ped/m²

Potential (ψ):
  min ≈ nan
  max ≈ nan

Domain area: 1.000000 m²


### 9.3 Check Mass Conservation

In [14]:
print("\nChecking mass conservation...\n")

# Compute ∇·(ρu) in domain
div_rho_u = div(gf_rho * u_cf)
mass_residual = Integrate(div_rho_u, mesh)

print(f"Mass conservation:")
print(f"  ∫ ∇·(ρu) dx = {mass_residual:.8e}")
print(f"  (should be ≈ 0 in steady state)")

if abs(mass_residual) < 1e-4:
    print(f"  ✓ Mass is conserved!")
else:
    print(f"  ⚠ Mass residual is large!")


Checking mass conservation...



AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'derivname'

## 10. Visualization

In [ ]:
print("\nGenerating visualizations...\n")

# Density field
Draw(gf_rho, mesh, "density")
print("  ✓ Density field (ρ)")

# Potential field
Draw(gf_psi, mesh, "potential")
print("  ✓ Potential field (ψ)")

# Velocity vector field
Draw(gf_u, mesh, "velocity", vectors={"grid_size": 30})
print("  ✓ Velocity vector field (u)")

print("\n✓ Visualization complete!")

## 11. Summary

### Key Results

- **Method**: Built-in Newton with automatic differentiation
- **System**: Steady-state Axthelm (Helmholtz + Continuity)
- **Coupling**: Symbolic velocity from trial functions
- **Convergence**: Check output above

### What We Learned

1. ✓ No manual Jacobian needed - NGSolve computes it automatically
2. ✓ Smooth nonlinearities essential for AD
3. ✓ Initial guess matters for convergence
4. ✓ Proper formulation: u computed from trial functions, not GridFunctions

### Next Steps

- Test different inflow rates
- Compare with time-stepping solution
- Vary regularization parameters
- Try different geometries

---

**Implementation**: Following `docs/12_BUILTIN_NEWTON_STEADY_STATE_PLAN.md`

**Reference**: Axthelm (2016) - "Finite Element Simulation of a Macroscopic Model for Pedestrian Flow"